# Assignment: SVD Preprocessing on MNIST with Logistic Regression

## Instructions:
In this assignment, you will apply **Singular Value Decomposition (SVD)** as a preprocessing step to the **MNIST dataset** and train a **logistic regression classifier**. You will compare the model performance and training time when using different levels of SVD for dimensionality reduction.

In this assignment, you will need to:
1. Load the MNIST dataset and normalize it.
2. Perform SVD and reduce the dimensions of the data.
3. Train a logistic regression model on the original and SVD-reduced data.
4. Measure and compare the training time and accuracy of the model with varying SVD components.
5. Plot the results and analyze how SVD impacts the performance and efficiency of the model.

***
Your tasks include:
1. Implement SVD algorithm. You are not allowed to directly use SVD implemented by other packages, but you may use functions in NumPy. (Part 2)
2. Explore the accuracy and time performance from different numbers of SVD components. (Part 4)
3. Visualize the accuracy, time performance and top 5 singular vectors in the dataset, analyze and explain which number of SVD component looks best to you? (Part 4,5&6) Hint: singular vectors should be reshaped to 28x28 images for visualization.
***
**Note that you may not import any other function or package.** Let's get started!


## Part 1: Load the MNIST dataset and preprocess the data

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score, classification_report

# Load MNIST dataset
print("Loading MNIST dataset...")
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target

# Normalize the data
X = X / 255.0

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Loading MNIST dataset...


## Part 2: Implement SVD for Dimensionality Reduction

In [3]:
print(X_train.shape)
print(X_test.shape)

(56000, 784)
(14000, 784)


In [4]:
def apply_svd_custom_optimized(X_train, X_test, n_components):
    # Center the data by subtracting the mean
    X_mean = np.mean(X_train, axis=0)
    X_train_centered = X_train - X_mean
    X_test_centered = X_test - X_mean
    
    # Compute the covariance matrix (X_train_centered.T @ X_train_centered) / n_samples
    covariance_matrix = np.dot(X_train_centered.T, X_train_centered) / X_train_centered.shape[0]
    
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)
    # Take only the real parts of eigenvalues and eigenvectors, otherwise get ComplexWarning on step 4
    eigenvalues = np.real(eigenvalues)
    eigenvectors = np.real(eigenvectors)
    
    # Use partial sorting to find the indices of the top n_components eigenvalues
    top_indices = np.argpartition(eigenvalues, -n_components)[-n_components:]
    top_indices = top_indices[np.argsort(eigenvalues[top_indices])[::-1]]
    
    # Get the top eigenvectors and corresponding eigenvalues
    top_eigenvectors = eigenvectors[:, top_indices]
    top_eigenvalues = eigenvalues[top_indices]
    
    # Project the train and test data onto the new lower-dimensional subspace
    X_train_reduced = np.dot(X_train_centered, top_eigenvectors)
    X_test_reduced = np.dot(X_test_centered, top_eigenvectors)
    
    return X_train_reduced, X_test_reduced, top_eigenvalues, top_eigenvectors

## Part 3: Train Logistic Regression and Measure Performance

In [7]:
# Function to train logistic regression and track training time
def train_logistic_regression(X_train, y_train, X_test, y_test):
    model = LogisticRegression(max_iter=1000, solver='saga', random_state=42)  # multi_class='multinomial'
    
    # Measure training time
    start_time = time.time()
    model.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy, training_time


# accuracy, training_time = train_logistic_regression(X_train, y_train, X_test, y_test)
# print(f"Accuracy: {accuracy:.4f}")
# print(f"Training Time: {training_time:.2f} seconds")

## Part 4: Experiment with Different Levels of SVD

Now, apply SVD with varying numbers of components and observe how the dimensionality reduction impacts the model's performance. Record both the accuracy and training time for each number of components.


In [41]:
svd_components = sorted(list({int((1.07 + x/500)**x) for x in range(40)}))

# Store the results
results = []

print("Training models with different levels of SVD preprocessing...")
for n_components in svd_components:
    print(f"Applying custom SVD with {n_components} components...")
    
    # Apply SVD to the training and test sets using the optimized SVD function
    X_train_svd, X_test_svd, singular_values, top_eigenvectors = apply_svd_custom_optimized(X_train, X_test, n_components)

    print('Training logistic regresion...')
    
    # Train the logistic regression model and get accuracy and training time
    accuracy, training_time = train_logistic_regression(X_train_svd, y_train, X_test_svd, y_test)
    
    # Store the result in the list
    results.append({
        "n_components": n_components,
        "accuracy": accuracy,
        "training_time": training_time
    })
    
    print(f"SVD components: {n_components}, Accuracy: {accuracy:.4f}, Training time: {training_time:.4f} seconds")


Training models with different levels of SVD preprocessing...
Applying custom SVD with 1 components...
Training logistic regresion...
SVD components: 1, Accuracy: 0.3069, Training time: 0.2951 seconds
Applying custom SVD with 2 components...
Training logistic regresion...
SVD components: 2, Accuracy: 0.4484, Training time: 0.4362 seconds
Applying custom SVD with 3 components...
Training logistic regresion...
SVD components: 3, Accuracy: 0.4718, Training time: 0.3563 seconds
Applying custom SVD with 4 components...
Training logistic regresion...
SVD components: 4, Accuracy: 0.5765, Training time: 0.4110 seconds
Applying custom SVD with 5 components...
Training logistic regresion...
SVD components: 5, Accuracy: 0.6796, Training time: 0.4444 seconds
Applying custom SVD with 6 components...
Training logistic regresion...
SVD components: 6, Accuracy: 0.7363, Training time: 0.4625 seconds
Applying custom SVD with 7 components...
Training logistic regresion...
SVD components: 7, Accuracy: 0.7

## Part 5: Visualize and Analyze the Results

Finally, plot the accuracy, training time as a function of the number of SVD components, and top 5 singular vectors. This will help you understand the trade-off between dimensionality reduction, accuracy, and model training time, and how SVD generally works. Hint: singular vectors should be reshaped to 28x28 images for visualization.


In [42]:
import pandas as pd

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results)

In [48]:
import plotly.graph_objects as go
import plotly.express as px

def plot_performance(results_df):
    """ Plots the accuracy and training time over number of SVD components with two y-axes
    """

    fig = go.Figure()

    # Plot accuracy on the left y-axis
    fig.add_trace(go.Scatter(
        x=results_df["n_components"], 
        y=results_df["accuracy"],
        mode="lines+markers",
        name="Accuracy",
        line=dict(color='blue'),
        yaxis="y1"  # left side
    ))

    # Plot training time on the right y-axis
    fig.add_trace(go.Scatter(
        x=results_df["n_components"], 
        y=results_df["training_time"],
        mode="lines+markers",
        name="Training Time (seconds)",
        line=dict(color='red', dash='dot'),
        yaxis="y2"  # right side
    ))

    # Update layout for dual y-axis
    fig.update_layout(
        title="Accuracy and Training Time vs Number of SVD Components",
        xaxis_title="Number of SVD Components",        
        yaxis=dict(
            title="Accuracy",
            titlefont=dict(color="blue"),
            tickfont=dict(color="blue"),
            range=[0, 1]
        ),
        yaxis2=dict(
            title="Training Time (seconds)",  # Right side y-axis
            titlefont=dict(color="red"),
            tickfont=dict(color="red"),
            anchor="x",  # Link this y-axis to the x-axis
            overlaying="y",  # Overlay it on top of the left y-axis
            side="right",
            range=[0, 100],
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    fig.show()

plot_performance(results_df)


def plot_singular_vectors(singular_vectors):
    """ Visualizes top 5 singular vectors as 28x28 images
    """
    top_5_singular_vectors = singular_vectors[:, :5].T
    reshaped_vectors = [vector.reshape(28, 28) for vector in top_5_singular_vectors]
    
    fig = px.imshow(np.concatenate(reshaped_vectors, axis=1), 
                    color_continuous_scale='gray', 
                    title=f"Top 5 Singular Vectors")
    
    fig.show()

plot_singular_vectors(top_eigenvectors)

## Part 6: Analyze / Conclusion 

YOUR ANSWER: 

In general, more components led to increased accuracy; however, there were clear diminishing returns, as accuracy plateued at very high number of components. Surprisingly, the model never exhibited overfitting; accuracy monotonically increased all the way up to the top tested value of 217 components even if ever so slightly. Training time was surprisingly noisy, but increased impressively close to linearly when running the below power law fit with a slope of 0.28 seconds per SVD component. Interestingly, using a reasonable ax/(b+x) + c fit suggests accuracy will asymptotically approach almost exactly 0.95 (1.04 - 0.09) for arbitrarily many components, but we achieve 0.92 accuracy in under a minute at 177 components and over 0.82 accuracy in under a second at 12 components, showing the dramatic tradeoff of increasing components and thereby increasing training time.

In [77]:
# best fits for above graph to sate my own curiosity

from scipy.optimize import curve_fit

def acc_fit(x, a, b, c):
    return a * x / (b + x) + c

def power_law_fit(x, a, b, c):
    return a * x ** b + c

def fit_curves_advanced(results_df):
    n_components = results_df["n_components"]
    
    # Exclude 0 values to avoid errors for logarithmic fit
    n_components_non_zero = n_components[n_components > 0]
    accuracy_non_zero = results_df["accuracy"].iloc[n_components_non_zero.index]
    time_non_zero = results_df["training_time"].iloc[n_components_non_zero.index]
    
    acc_params, _ = curve_fit(acc_fit, n_components_non_zero, accuracy_non_zero)
    a, b, c = acc_params
    print(f'acc ~ {a}*x/({b} + x) + {c}')

    initial_guess = [1, 0.01, 1]
    exp_params, _ = curve_fit(power_law_fit, n_components_non_zero, time_non_zero, p0=initial_guess, bounds=(0, [1000, 1, 1000]))
    a, b, c = exp_params
    print(f'time ~ {a}*x**{b} + {c}')

    # Generate fitted values
    n_components_range = np.linspace(min(n_components), max(n_components), 100)
    
    fitted_accuracy_log = acc_fit(n_components_range, *acc_params)
    fitted_time_exp = power_law_fit(n_components_range, *exp_params)

    # Plot the data with fitted curves
    fig = go.Figure()

    # Plot original accuracy on the left y-axis (solid line)
    fig.add_trace(go.Scatter(
        x=n_components, 
        y=results_df["accuracy"],
        mode="lines+markers",
        name="Accuracy",
        line=dict(color='blue', dash='solid'),
        yaxis="y1"
    ))

    # Plot logarithmic fit for accuracy
    fig.add_trace(go.Scatter(
        x=n_components_range, 
        y=fitted_accuracy_log,
        mode="lines",
        name="Accuracy 1/1+x Fit",
        line=dict(color='blue', dash='dash'),
        yaxis="y1"
    ))

    # Plot original training time on the right y-axis (dotted line)
    fig.add_trace(go.Scatter(
        x=n_components, 
        y=results_df["training_time"],
        mode="lines+markers",
        name="Training Time (seconds)",
        line=dict(color='red', dash='dot'),
        yaxis="y2"
    ))

    # Plot exponential fit for training time
    fig.add_trace(go.Scatter(
        x=n_components_range, 
        y=fitted_time_exp,
        mode="lines",
        name="Training Time Exp Fit",
        line=dict(color='green', dash='dot'),
        yaxis="y2"
    ))

    # Update layout for dual y-axis with custom ranges
    fig.update_layout(
        title="Accuracy and Training Time vs Number of SVD Components with Logarithmic and Exponential Fits",
        xaxis_title="Number of SVD Components",
        
        yaxis=dict(
            title="Accuracy",
            titlefont=dict(color="blue"),
            tickfont=dict(color="blue"),
            range=[0, 1]
        ),
        
        yaxis2=dict(
            title="Training Time (seconds)",
            titlefont=dict(color="red"),
            tickfont=dict(color="red"),
            anchor="x",
            overlaying="y",
            side="right",
            range=[0, 100]
        ),
        
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    fig.show()

# Call the advanced curve fitting function with the DataFrame results_df
fit_curves_advanced(results_df)


acc ~ 1.0440370364490723*x/(1.791869154266387 + x) + -0.09654091866991857
time ~ 0.28232614996355965*x**0.99999999999737 + 9.035274860442381e-15


In [58]:
results_df

,n_components,accuracy,training_time
0,1,0.306857,0.295077
1,2,0.448357,0.436165
2,3,0.471786,0.356321
3,4,0.576500,0.411016
4,5,0.679643,0.444396
5,6,0.736286,0.462523
6,7,0.761857,0.454060
7,8,0.787071,0.509744
8,9,0.789286,0.663166
9,10,0.803500,0.738182
